In [2]:
from loading_data import queryDB
import pandas as pd

In [3]:
import json
import numpy as np
import copy
import os

config_path = 'config.json'

with open(config_path) as config_buffer:
   config = json.loads(config_buffer.read())

model_path = config['model_weights']
num_concepts = len(config['conceptids'])
class_map_file = config['class_map']
concepts = config['conceptids']
classmap = pd.read_csv(class_map_file, header=None).to_dict()[0]

In [4]:
counts = queryDB('''
    select videoid, count(*) as total, 
        sum(case when conceptid=1629 then 1 else 0 end) As count1629, 
        sum(case when conceptid=1210 then 1 else 0 end) As count1210, 
        sum(case when conceptid=236 then 1 else 0 end) As count236, 
        sum(case when conceptid=383 then 1 else 0 end) As count383, 
        sum(case when conceptid=1133 then 1 else 0 end) As count1133 
    from annotations where userid != 17 and conceptid in (1629, 1210, 236, 383, 1133)
    group by videoid 
    order by total desc;
       ''')

counts.head(20)
       

,videoid,total,count1629,count1210,count236,count383,count1133
0,37,3916,0,4,2885,1024,3
1,39,2186,0,0,1077,1108,1
2,38,1032,0,0,660,371,1
3,34,941,0,2,860,78,1
4,24,459,0,0,459,0,0
5,32,420,0,61,80,273,6
6,33,340,0,16,208,109,7
7,86,331,0,40,143,148,0
8,98,309,0,29,181,97,2
9,35,288,0,9,221,58,0


In [5]:
queryDB("select * from videos where filename='DocRicketts-0701_20141216T212020Z_00-48-12-26TC_h264.mp4'")


,id,filename,gpsstart,gpsstop,starttime,endtime,startdepth,enddepth,description
0,8,DocRicketts-0701_20141216T212020Z_00-48-12-26T...,"(36.798367,-122.108671)","(36.798215,-122.108195)",2014-12-16 21:20:20,2014-12-16 21:36:20,217,149,


In [132]:
annotations = queryDB('select * from annotations where videoid=86 and userid!=17')

fps = 29.97002997002997
annotations['frame_num'] = np.rint(annotations['timeinvideo'] * fps).astype(int)
annotations.head()

In [134]:
# Get the IOU value for two different annotations
def compute_overlap(annotationA, annotationB):
    # if there is no overlap in x dimension
    if ((annotationB.x2 - annotationA.x1) < 0) or ((annotationA.x2 - annotationB.x1) < 0):
        return 0
    # if there is no overlap in y dimension
    if ((annotationB.y2 - annotationA.y1) < 0) or ((annotationA.y2 - annotationB.y1) < 0):
        return 0
    
    areaA = (annotationA.x2-annotationA.x1) * (annotationA.y2-annotationA.y1)
    areaB = (annotationB.x2-annotationB.x1) * (annotationB.y2-annotationB.y1)

    width = min(annotationA.x2,annotationB.x2) - min(annotationA.x1,annotationB.x1)
    height = min(annotationA.y2,annotationB.y2) - min(annotationA.y1,annotationB.y1)
    
    area_intersect = height * width
    iou = area_intersect / (areaA + areaB - area_intersect)
    
    return iou

In [135]:
# GET RESULTS, RENAME ID COLUMNS in RESULTS

results = pd.read_csv('results86.csv')
results.columns = ['id', 'x1', 'y1', 'x2', 'y2', 'label', 'confidence', 'objectid', 'frame_num']

#results.head()

In [136]:
# Number of predicted objects
len(results.objectid.value_counts())

2372

In [137]:
# Number of test annotations for our concepts
len(annotations[[(a in concepts) for a in annotations.conceptid]])

331

In [138]:
# Limit Results based on object max frame confidence
def conf_limit_objects(pred, threshold):
    max_conf = pd.DataFrame(pred.groupby('objectid').confidence.max())
    above_thresh = max_conf[max_conf.confidence > 0.30].index
    return pred[[(obj in above_thresh) for obj in pred.objectid]]
    

In [139]:
results = conf_limit_objects(results, 0.30)

In [140]:
# Number of predicted objects post thresholding
len(results.objectid.value_counts())

98

In [141]:
# REVISED LABEL TRANSFER USING HIGHEST SUM OF CONFIDENCES

# conf_sums = results.groupby(['objectid', 'label']).confidence.sum()
# df = conf_sums.reset_index()
# df2 = df.loc[df.groupby('objectid').label.idxmax()]
# labels = dict(zip(df2.objectid, df2.label))
# results['label'] = results.objectid.apply(lambda x: concepts[int(labels[x])])
# results['conceptid'] = results['label']


In [142]:
# REVISED LABEL TRANSFER USING HIGHEST MAXIMUM CONFIDENCE

# conf_max = results.groupby(['objectid', 'label']).confidence.max()
# df = conf_max.reset_index()
# df2 = df.loc[df.groupby('objectid').label.idxmax()]
# labels = dict(zip(df2.objectid, df2.label))
# results['label'] = results.objectid.apply(lambda x: concepts[int(labels[x])])
# results['conceptid'] = results['label']


In [143]:
# Add the majority concept id to all frames

label = None
objects = results.groupby(['objectid'])
for oid, result in objects:
    scores = {}
    # SHOULD THIS BE A WEIGHTED SUM??
    for k , label in result.groupby(['label']):
        scores[k] = label.confidence.mean() # MEAN / MAX / SUM?
    idmax = max(scores.keys(), key=(lambda k: scores[k]))
    results.loc[results.objectid == oid, 'label'] = idmax
results['label'] = results['label'].apply(lambda x: concepts[int(x)])
results['conceptid'] = results['label']

In [144]:
#results.groupby('objectid').label.value_counts()

In [145]:
# Limit results based on tracked object length ( > 30 frames)
def length_limit_objects(pred, threshold):
    obj_len = pred.groupby('objectid').conceptid.value_counts()
    len_thresh = obj_len[obj_len > threshold]
    return pred[[(obj in len_thresh) for obj in pred.objectid]] 

In [146]:
results = length_limit_objects(results, 15)

In [147]:
len(results.objectid.value_counts())

93

In [148]:
# test counts
def get_counts(results):
    grouped = results.groupby(['objectid']).label.mean().reset_index()
    counts = grouped.groupby('label').count()
    counts.columns = ['pred_num']
    groundtruth_counts = pd.DataFrame(annotations.groupby('conceptid').id.count())
    groundtruth_counts.columns = ['true_num']
    return pd.concat((counts, groundtruth_counts), axis=1, join='outer').fillna(0)


In [149]:
concept_counts = get_counts(results)

In [151]:
def resize(row):
    new_width = 640
    new_height = 480
    x_ratio = (row.videowidth / new_width)
    y_ratio = (row.videoheight / new_height)
    row.videowidth = new_width
    row.videoheight = new_height
    row.x1 = row.x1 / x_ratio
    row.x2 = row.x2 / x_ratio
    row.y1 = row.y1 / y_ratio
    row.y2 = row.y2 / y_ratio
    return row

In [152]:
# RESIZE THE ANNOTATIONS
annotations = annotations.apply(resize, axis=1)

In [153]:
# pred_frames = results.frame_num.unique()
# test_frames = annotations.frame_num.unique()
# test_frames[[(val in pred_frames) for val in test_frames]]
# test_frames

In [154]:
def score_predictions(validation, predictions, iou_thresh, concepts, fps):
    
    # Maintain a set of predicted objects to verify
    detected_objects = []
    obj_map = predictions.groupby('objectid', sort=False).conceptid.max()
    
    # group predictions by video frames
    predictions = predictions.groupby('frame_num', sort=False)
    predictions = [df for _, df in predictions]
    
    # mapping frames to predictions index
    frame_data = {}
    for i, group in enumerate(predictions):
        frame_num = group.iloc[0]['frame_num']
        frame_data[frame_num] = i
    
    # group validation annotations by frames
    validation = validation.groupby('frame_num', sort=False)
    validation = [df for _, df in validation]
    
    # initialize counters for each concept
    true_positives = dict(zip(concepts,[0] * len(concepts)))
    false_positives = dict(zip(concepts,[0] * len(concepts)))
    false_negatives = dict(zip(concepts,[0] * len(concepts)))
    
    # get true and false positives for each frame of validation data
    for group in validation:
        try: # get corresponding predictions for this frame
            frame_num = group.iloc[0]['frame_num']
            predicted = predictions[frame_data[frame_num]]
        except:
            continue # False Negatives already covered
            
        detected_truths = dict(zip(concepts, [0] * len(concepts)))
        for index, truth in group.iterrows():
            for index, prediction in predicted.iterrows():
                if (prediction.conceptid == truth.conceptid
                        and compute_overlap(truth, prediction) > iou_thresh
                        and prediction.objectid not in detected_objects):
                    detected_objects.append(prediction.objectid)
                    true_positives[prediction.conceptid] += 1
                    detected_truths[prediction.conceptid] += 1
                    
        # False Negatives (Missed ground truth predicitions)
        counts = group.conceptid.value_counts()
        for concept in concepts:
            count = counts[concept] if (concept in counts.index) else 0
            false_negatives[concept] += count - detected_truths[concept]
    
    # False Positives (No ground truth prediction at any frame for that object)
    undetected_objects = set(obj_map.index) - set(detected_objects)
    for obj in undetected_objects:
        concept = obj_map[obj]
        false_positives[concept] += 1
    
    metrics = pd.DataFrame()
    for concept in concepts:
        TP = true_positives[concept]
        FP = false_positives[concept]
        FN = false_negatives[concept]
        precision = TP / (TP + FP) if (TP + FP) != 0 else 0
        recall = TP / (TP + FN) if (TP + FN) != 0 else 0
        f1 = (2*recall*precision / (precision+recall)) if (precision+recall) != 0 else 0
        metrics = metrics.append([[concept, TP, FP, FN, precision, recall, f1]])
    metrics.columns = ['conceptid', 'TP', 'FP', 'FN', 'Precision', 'Recall', 'F1']
    return metrics


In [155]:
fps = 29.97002997002997
metrics = score_predictions(test_annotations, results, 0.25, concepts, fps)


In [156]:
#metrics.set_index('conceptid')

In [157]:
metrics.set_index('conceptid').join(concept_counts)

,TP,FP,FN,Precision,Recall,F1,pred_num,true_num
conceptid,,,,,,,,
1629,0,0,0,0.000000,0.000000,0.000000,NaN,NaN
1210,1,2,12,0.333333,0.076923,0.125000,3.0,40.0
236,24,23,33,0.510638,0.421053,0.461538,47.0,143.0
383,39,4,22,0.906977,0.639344,0.750000,43.0,148.0
1133,0,0,0,0.000000,0.000000,0.000000,NaN,NaN


In [ ]:
# Aim for 90% precision, at the sacrifice of 50% recall

In [ ]:
# make a threshold for the bounding box size of the annotation
# for the maximum confidence annotation in each object
# maybe 10x10 pixels